Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_psi_t_var_formula">_psi_t_var_formula</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, c, k, m, n, t, P, defaults, Function
from proveit.linear_algebra import ScalarMult, TensorProd
from proveit.logic import Equals, InSet
from proveit.numbers import (
        zero, one, two, i, e, pi, Add, Exp, Less, LessEq, Mult, frac, Neg, subtract)
from proveit.numbers import Complex, Interval, Natural
from proveit.numbers.number_sets.natural_numbers import fold_forall_natural_pos
from proveit.physics.quantum import Ket
from proveit.physics.quantum.QPE import _phase, _phase_is_real, _psi_t_def

In [ ]:
%proving _psi_t_var_formula

In [ ]:
defaults.assumptions = _psi_t_var_formula.all_conditions()

Keep the $2$ in $2 \pi i$ from combining with powers of $2$:

In [ ]:
Exp.change_simplification_directives(factor_numeric_rational=True)

In [ ]:
# the induction theorem for positive naturals
fold_forall_natural_pos

In [ ]:
# instantiate the induction theorem
induction_inst = fold_forall_natural_pos.instantiate(
    {Function(P,t):_psi_t_var_formula.instance_expr, m:t, n:t})

### Some Related Properties and Definitions Needed for Later Processing
Mainly: some domains and orderings. Notice that throughout this section devoted to the induction proof, $t$ is a _variable_, not a literal.

In [ ]:
# used when processing products involving the phase phi
_phase_is_real

In [ ]:
two_pow_t_var_less_one = subtract(Exp(two, t), one)

In [ ]:
Exp(two, t).deduce_bound(Less(t, Add(t, one)))

### Base Case

In [ ]:
base_case = induction_inst.antecedent.operands[0]

We have $|\psi_{t}\rangle$ defined as a tensor product (the result of the first phase of the quantum circuit, and the LHS of Nielsen & Chuang's identity 5.20 on pg 222):

In [ ]:
_psi_t_def

In [ ]:
_psi_t_as_tensor_prod = _psi_t_def.instantiate()

For $\psi'_{1}$, we prove a useful equality then instantiate the `psi_t_def` with $t=1$:

In [ ]:
psi_1_def = _psi_t_def.instantiate({t:one})

Then show that the summation formula also gives the same qbit result

In [ ]:
sum_0_to_1 = base_case.rhs

In [ ]:
sum_0_to_1_processed_01 = sum_0_to_1.inner_expr().operands[1].partitioning_first()

In [ ]:
# finish off the Base Case
base_case_jdgmt = sum_0_to_1_processed_01.sub_left_side_into(psi_1_def)

### Inductive Step

In [ ]:
inductive_step = induction_inst.antecedent.operands[1]

In [ ]:
defaults.assumptions = defaults.assumptions + inductive_step.conditions.entries

First, partition the summation:
$\sum_{k=0}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} = \sum_{k=0}^{2^{t}-1} e^{2\pi i \varphi k} |k\rangle_{t+1} + \sum_{k=2^{t}}^{2^{t+1}-1} e^{2\pi i \varphi k} |k\rangle_{t+1}$

In [ ]:
summation_partition_01 = (
    inductive_step.instance_expr.rhs.operands[1]
    .partitioning(two_pow_t_var_less_one))

Then shift the second summation of that partition, so that the two summations then have the same index domain:

In [ ]:
summation_partition_02 = (summation_partition_01.inner_expr().rhs.
                          operands[1].shift(Neg(Exp(two, t))))

We want to rewrite the summand of that 2nd summation on the rhs now by (1) expanding the exponential term and (2) rewriting the $|k+2^t{\rangle}_{t+1}$ ket as $|1\rangle \otimes |k{\rangle}_t$. This takes a little work.

In [ ]:
rhs_2nd_sum = summation_partition_02.rhs.operands[1]

In [ ]:
summand_processed_01 = rhs_2nd_sum.summand.inner_expr().operands[0].exponent.distribution(
    4, assumptions=[*defaults.assumptions,
                    rhs_2nd_sum.condition])

In [ ]:
summand_processed_02 = (
    summand_processed_01.inner_expr().rhs.operands[0]
    .exponent_separate(
        assumptions=[*defaults.assumptions,
                     InSet(k, Interval(zero, subtract(Exp(two, t), one)))]))

In [ ]:
# Avoid recombining these via auto-simplification
defaults.preserved_exprs = set([summand_processed_02.rhs.scalar])

In [ ]:
from proveit.physics.quantum.algebra import prepend_num_ket_with_one_ket
prepend_num_ket_with_one_ket

In [ ]:
prepend_num_ket_with_one_ket_inst = prepend_num_ket_with_one_ket.instantiate(
        {n: t, k: k},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
summand_processed_03 = (
    summand_processed_02.inner_expr().rhs.operands[1]
    .substitute(prepend_num_ket_with_one_ket_inst))

In [ ]:
# reminder of summation_partition_03
summation_partition_02

In [ ]:
summation_partition_04 = (
    summation_partition_02.inner_expr().rhs.operands[1].summand.substitute(
        summand_processed_03))

Also process the summand in the 1st sum on the rhs, converting the ket $|k\rangle_{t+1}$ to the tensor product $|0\rangle \otimes |k\rangle_{t}$:

In [ ]:
# pull up the relevant NumKet theorem
from proveit.physics.quantum.algebra import prepend_num_ket_with_zero_ket
prepend_num_ket_with_zero_ket

In [ ]:
# instantiate the theorem for our specific case
prepend_num_ket_with_zero_ket_inst = prepend_num_ket_with_zero_ket.instantiate(
        {n: t, k: k},
        assumptions=[*defaults.assumptions, InSet(k, Interval(zero, subtract(Exp(two, t), one)))])

In [ ]:
# use our instantiation to substitute
summation_partition_05 = (
    summation_partition_04.inner_expr().rhs.operands[0].summand.scaled.substitute(
    prepend_num_ket_with_zero_ket_inst))

We also then want to:

(1) pull the tensor product out of the 2nd summation, and

(2) pull the non-$k$-dependent exponential factor out of the 2nd summation.

In [ ]:
# for convenience:
temp_factors = summation_partition_05.rhs.operands[1].summand.operands[0]

In [ ]:
from proveit.linear_algebra import VecSpaces
from proveit.numbers import Real
VecSpaces.default_field=Complex
summation_partition_06 = summation_partition_05.inner_expr().rhs.operands[1].factors_extract()

In [ ]:
summation_partition_07 = (
    summation_partition_06.inner_expr().rhs.operands[0].factors_extract())

Now factor out the common vector summation factor in the two terms of the rhs. We do this using our VecAdd.factorization() method after we quickly re-associate terms in the far right-hand side vector.

In [ ]:
temp_summation = summation_partition_07.rhs.operands[1].scaled.operands[1]
desired_expr = TensorProd(ScalarMult(temp_factors.operands[1], Ket(one)), temp_summation)

In [ ]:
summation_partition_08 = (
    desired_expr.scalar_factorization(idx=0)
    .sub_left_side_into(summation_partition_07.inner_expr().rhs.operands[1]))

Now as promised we utilize the VecAdd.factorization() method. We need to let it know we're dealing with vectors over the field of Complex numbers and we want to “pull” the factor to the right:

In [ ]:
temp_factor = summation_partition_08.rhs.operands[0].operands[1]
summation_partition_09 = (
    summation_partition_08.inner_expr().rhs.factor(
        temp_factor, pull='right', field=Complex))

Reintroduce the scalar coefficient and put the expression in a desired form:

In [ ]:
summation_partition_10 = inductive_step.instance_expr.rhs.inner_expr().scaled.substitution(summation_partition_09)

In [ ]:
# represent the desired (distributed scalar-product) outcome:
desired_expr = (
    TensorProd(
        ScalarMult(frac(one, Exp(two, frac(one, two))), summation_partition_10.rhs.scaled.operands[0]),
        ScalarMult(frac(one, Exp(two, frac(t, two))), summation_partition_10.rhs.scaled.operands[1])
    ))

In [ ]:
# let Prove-It prove the equality:
distrib_equality = Equals(summation_partition_10.rhs, desired_expr).prove()

In [ ]:
# now use that equality in our earlier work:
conclusion_01 = distrib_equality.sub_right_side_into(summation_partition_10)

In [ ]:
# Recall our inductive hypothesis:
for item in defaults.assumptions:
    if isinstance(item, Equals):
        inductive_hypothesis = item
inductive_hypothesis

In [ ]:
# use the inductive hypothesis:
conclusion_02 = inductive_hypothesis.sub_left_side_into(conclusion_01)

The RHS of conclusion_02 above should be equal to $|\psi_{t+1}\rangle$. Let's see how to get there. Ideally we could use the next three cells to show the rhs of the equality in `conclusion_02` is equal to $|\psi_{t+1}\rangle$

In [ ]:
# reminder
_psi_t_def

In [ ]:
psi_t_plus_one_as_tensor_prod = _psi_t_def.instantiate({t:Add(t, one)})

Now this `substitute()` step will also manage to pull the scalar coefficients out to the front in the process:

In [ ]:
conclusion_03 = conclusion_02.inner_expr().rhs.operands[1].substitute(
        _psi_t_as_tensor_prod)

Now we partition the tensor product expression of $\psi_{t+1}$, so we can eventually show that it is equal to the rhs of conclusion_03.

In [ ]:
psi_t_plus_one_as_tensor_prod.rhs

In [ ]:
# # we can partition the operands in the psi_{t+1} tensor prod expression
psi_t_plus_one_as_tensor_prod_02 = (
    psi_t_plus_one_as_tensor_prod.inner_expr().rhs.operands[0].split(t))

In [ ]:
conclusion_03.inner_expr().rhs.substitute(psi_t_plus_one_as_tensor_prod_02.derive_reversed())

In [ ]:
# recall our earlier instantiation of the induction theorem:
induction_inst

We have proven both pieces of the antecedent (_i.e._, the base case and inductive case) of the instantiated induction theorem `induction_inst`, so we can now derive the consequent:

In [ ]:
induction_inst.derive_consequent()

In [ ]:
%qed